In [1]:
import pandas as pd
import numpy as np
import xlsxwriter

In [2]:
# Min temp data basefile
mnfile = "../Weather Variables/MinTemp/" 
# Max temp data basefile
mxfile = "../Weather Variables/MaxTemp/" 
# Precipiation data basefile
pfile = "../Weather Variables/Ppt/" 

In [3]:
def findMaxMin(db, code):
    if code == 'max':
        return db.max(axis=1)
    else:
        return db.min(axis=1)

In [4]:
def summarizeWeatherData(var):
    if var == 'maxT':
        bfile = mxfile
        code = 'max'
    elif var == "minT":
        bfile = mnfile
        code = 'min'
    else:
        bfile = pfile
        return pd.read_excel(bfile+'PrecipitationRecord1981_2013.xlsx')

    tdb = pd.read_excel(bfile+str(1991)+'.xlsx')
    merged = tdb[['UID']]
    for year in np.arange(33)+1981:
        db = pd.read_excel(bfile+str(year)+'.xlsx')
        adb = db[[u'jan',  u'feb',  u'mar',  u'apr', u'may',  u'jun',  u'jul',  u'aug',  u'sep',  u'oct',  u'nov',  
                  u'dec']]
        req = findMaxMin(adb, code)
#         print np.array(req)[0:3]
        req.name = str(year)
        merged = merged.join(req)
    return merged

In [5]:
minTemp = summarizeWeatherData('minT')

In [6]:
maxTemp = summarizeWeatherData('maxT')

In [7]:
Ppt = summarizeWeatherData('ppt')

In [8]:
Ppt.head()

,UID,Longitude,Latitude,1981,1982,1983,1984,1985,1986,1987,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,222,-121.802218,47.860322,1690.469971,250.389999,1664.189941,1833.619995,1204.719971,1657.739990,1117.500000,...,1611.349976,1434.619995,1799.180054,1725.300049,1687.619995,1554.349976,1633.670044,1840.020020,2006.119995,1656.489990
1,540,-121.252570,47.715980,2157.649902,477.429993,2289.860107,2530.840088,1734.770020,2106.399902,1655.130005,...,1964.030029,1785.910034,2404.729980,2059.379883,2107.500000,2045.839966,2090.399902,2544.739990,2465.479980,1968.800049
2,2431,-118.301093,47.646535,337.529999,36.470001,501.670013,363.320007,228.970001,317.239990,319.609985,...,306.170013,359.350006,424.649994,236.389999,248.000000,287.329987,481.829987,287.309998,469.309998,258.200012
3,704,-121.000382,47.774584,1980.089966,483.109985,2204.989990,2411.659912,1679.930054,1992.989990,1580.770020,...,1923.250000,1703.020020,2439.649902,2022.010010,2015.349976,2088.709961,2009.209961,2498.659912,2463.620117,1904.589966
4,776,-120.868954,47.787672,1099.699951,336.149994,1390.640015,1228.729980,845.900024,1152.069946,951.090027,...,849.609985,891.830017,1454.770020,1019.559998,1037.709961,1089.160034,1206.229980,1441.949951,1611.650024,992.859985


In [12]:
minTemp.index = np.array(minTemp.UID)
maxTemp.index = np.array(maxTemp.UID)
Ppt.index = np.array(Ppt.UID)

In [9]:
segments = np.unique(np.array(Ppt.UID))
msegs = np.tile(segments, 33)
sorted_segments = np.sort(msegs)
print len(sorted_segments) == len(segments) *33
years = np.arange(33)+1981
myears = np.tile(years, len(segments))
print len(sorted_segments) == len(myears)

True
True


In [10]:
segDic = {'ID': sorted_segments, 'Year': myears}
cumdf = pd.DataFrame(segDic)

In [11]:
print len(cumdf)

294954


In [16]:
def createCumDF(cumdf, df): 
    tcumdf = cumdf.copy()
    for x in segments:
        for year in years:
            tcumdf[tcumdf.ID == x].loc[:, str(year)] = df.loc[x, str(year)]
    return tcumdf

In [ ]:
cumPpt = createCumDF(cumdf, Ppt)

In [10]:
df = pd.read_excel('../modelData/cdata_cim.xlsx')

In [11]:
len(df[df.sYear < 2014]) == len(df)

True

In [12]:
def popPPT(row):
    index = row['uid']
    year  = row['sYear']
    val   = str(year)
    return Ppt.loc[index, val]
df.loc[:, 'PPT'] = df.apply(popPPT, axis = 1)

In [13]:
def popMinT(row):
    index = row['uid']
    year  = row['sYear']
    val   = str(year)
    return minTemp.loc[index, val]
df.loc[:, 'MinTemp'] = df.apply(popMinT, axis = 1)

In [14]:
def popMaxT(row):
    index = row['uid']
    year  = row['sYear']
    val   = str(year)
    return maxTemp.loc[index, val]
df.loc[:, 'MaxTemp'] = df.apply(popMaxT, axis = 1)

In [15]:
df.head()

,sYear,Rutting,IRI,PSC,IntID,SR,Begin ARM,End ARM,uid,ID_y,...,ESALs,Contract History ID,STcode,Latitude,Longitude,ICAO,FTCycles,PPT,MinTemp,MaxTemp
0,1981,0.236,157.732,81,7,2,58.81,58.85,589,6d84b3f5-0725-4999-84e6-116b5c9b2e4e,...,33850.287773,6d84b3f5-0725-4999-84e6-116b5c9b2e4e,BA,47.712741,-121.148306,KESW,109,2167.459961,-4.35,25.139999
1,1981,0.236,156.465,52,28,2,29.40,29.45,295,96f2f5bb-a993-44d8-b9fc-e98ed55b333d,...,59474.398810,96f2f5bb-a993-44d8-b9fc-e98ed55b333d,Other,47.840245,-121.665412,KAWO,0,2071.070068,0.51,25.990000
2,1981,0.236,139.362,75,64,2,58.27,58.31,584,362e294a-0a07-43e7-b3f3-bfba58443fad,...,35074.861747,362e294a-0a07-43e7-b3f3-bfba58443fad,BA,47.712741,-121.148306,KESW,109,2167.459961,-4.35,25.139999
3,1981,NaN,126.059,0,473,2,312.24,312.34,3124,276269f6-c706-44fe-becb-1e2caf22cdae,...,69707.873892,276269f6-c706-44fe-becb-1e2caf22cdae,Other,48.011505,-117.349583,KSFF,20,557.729980,-4.84,31.610001
4,1981,0.236,102.621,75,514,2,129.28,129.38,1293,171a9c25-bffa-4ab9-ab96-85fdb6fc8b4a,...,116417.204918,171a9c25-bffa-4ab9-ab96-85fdb6fc8b4a,Other,47.498492,-120.297298,KEAT,24,292.450012,-4.83,32.799999


In [16]:
df.to_excel('../modelData/CIMdataset2.xlsx')